<a href="https://colab.research.google.com/github/prerna-khanna/Air-Cognizer/blob/master/Feature_Extraction_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The given script was used for extracting features which are dependent upon the concentration of PM2.5 particles suspended in air and hence can be used for estimating the same. There are in total 6 image features:

1. Blue 

2. Transmission 

3. Gradient 

4. Root mean square of image

5. Constrast of ROI

6. Entropy

## Importing Libraries

In [8]:
import cv2;
import math;
import numpy as np;
import os, os.path;
from numpy import unique;
from scipy.stats import entropy as scipy_entropy
import xlwt;
from skimage import io,color
from google.colab import files
from google.colab.patches import cv2_imshow
uploaded = files.upload()

print("Imported LIbraries Successfully")

Saving 394.jpg to 394.jpg
Imported LIbraries Successfully


## Function for splitting color channels 


In [0]:
def Split(src):
    b,g,r = cv2.split(src);
    green = np.matrix(g);
    red = np.matrix(r);
    blue = np.matrix(b);
    print('rgb mean =',np.mean(r+g+b));

    lab = color.rgb2lab(src)
    l,a,c = cv2.split(lab);
    x = np.matrix(l);
    y = np.matrix(a);
    z = np.matrix(c);
    print('lab mean =',np.mean(x+y+z));

## Estimating Entropy
**Input** is the normalised grayscale image and a positive number which serves as base of logarithm in entropy estimation.

**Returns** entropy as a floating point number. 

In [0]:
def Entropy(normal, base = 2):  #2 is for log to the base 2

    _, counts = unique(normal, return_counts=True)
    return scipy_entropy(counts, base=base)

## Estimating Contrast of ROI
**Input** is the grascale image of ROI (for contrast ROI is non sky part) and returns the **contrast** as a floating point number. 

In [0]:
def Roi_Contrast(signal): 
    a = np.array(signal)
    normal = np.divide(a,255)
    b = np.mean(normal,dtype=np.float64)
    height,width = normal.shape[:2]

    for ix,iy in np.ndindex(normal.shape):
      
            c = normal[ix,iy]
            d = c-b
            d = d*d
            e = d
            d = d+e        
                        
    den = width*height
    rms = d/den
    return (math.sqrt(rms))    



## Estimating Gradient
**Input** is grayscale image and gradient is estimated of the ROI (for gradient ROI is the sky regoi of the image) and function returns **positive floating point number**. 

In [0]:
def Gradient(roi1):
    white = np.array([255,255,255])
    lowerBound = np.array([200,200,200])
    mask = cv2.inRange(roi1, lowerBound, white)
    laplacian = cv2.Laplacian(mask, cv2.CV_64F);
    gradient = cv2.split(laplacian);  
    
    return abs(np.mean(gradient))

##Blue color estimatoin 
**Input** is RGB image and outputs the **RMS of blue** color of sky. 

In [0]:
def Blue(im):
    b,g,r = cv2.split(im);
    blue = np.matrix(b);
    return np.mean(blue)
  

# RMS of the image
The **input** is the grayscale image and outputs the RMS as floating point number. 

In [0]:
def RMS(signal): 
    a = np.array(signal)
    normal = np.divide(a,255)
    b = np.mean(normal,dtype=np.float64)
    height,width = normal.shape[:2]

    for ix,iy in np.ndindex(normal.shape):
      
            c = normal[ix,iy]
            d = c-b
            d = d*d
            e = d
            d = d+e        
                        
    den = width*height
    rms = d/den
    return math.sqrt(rms)